In [1]:
pip install -q tensorflow-model-optimization

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import os
import zipfile
import tempfile
import math
import pathlib

In [3]:
from google.colab import drive
drive.mount ('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [4]:
train = pd.read_csv('/drive/My Drive/sign_mnist_train.csv')
test = pd.read_csv('/drive/My Drive/sign_mnist_test.csv')

In [5]:
x_train = train.drop(['pixel1'], axis=1).values
y_train = train['label'].values
x_test = test.drop(['pixel1'], axis=1).values
y_test = test['label'].values
x_train = x_train / 255
x_test = x_test / 255

In [6]:
x_train = np.array(x_train)
x_test = np.array(x_test)
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)

In [7]:
model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(28, 28, 1)),
        keras.layers.Reshape(target_shape=(28, 28, 1)),
        keras.layers.Conv2D(filters=16, kernel_size=(3,3), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=32, kernel_size=(6,6), activation=tf.nn.relu, padding='same'),
        
        keras.layers.Conv2D(filters=64, kernel_size=(12,12), activation=tf.nn.relu, padding='same'),

        keras.layers.MaxPooling2D(pool_size=(2,2)),
        keras.layers.Flatten(),
        keras.layers.Dense(1024, activation='relu'),
        keras.layers.Dropout(0.20),
        keras.layers.Dense(25, activation='softmax')                  
])

In [8]:
model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

In [9]:
model.fit(x_train, y_train, epochs=10,validation_data=(x_test, y_test))

Epoch 1/10
858/858 [==============================] - 23s 8ms/step - loss: 0.5106 - accuracy: 0.8437 - val_loss: 0.3581 - val_accuracy: 0.9134
Epoch 2/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0237 - accuracy: 0.9924 - val_loss: 0.3389 - val_accuracy: 0.9353
Epoch 3/10
858/858 [==============================] - 7s 8ms/step - loss: 5.1090e-04 - accuracy: 1.0000 - val_loss: 0.3439 - val_accuracy: 0.9463
Epoch 4/10
858/858 [==============================] - 7s 8ms/step - loss: 3.8281e-05 - accuracy: 1.0000 - val_loss: 0.3596 - val_accuracy: 0.9455
Epoch 5/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0235 - accuracy: 0.9929 - val_loss: 0.4603 - val_accuracy: 0.8985
Epoch 6/10
858/858 [==============================] - 7s 8ms/step - loss: 0.0067 - accuracy: 0.9982 - val_loss: 0.3236 - val_accuracy: 0.9374
Epoch 7/10
858/858 [==============================] - 7s 8ms/step - loss: 9.9717e-05 - accuracy: 1.0000 - val_loss: 0.3501 - val_accuracy: 

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        294976    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [11]:
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
print('\nTest accuracy:', test_acc)

225/225 - 1s - loss: 0.3335 - accuracy: 0.9445

Test accuracy: 0.9445064067840576


In [12]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
print('Saving model to: ', keras_file)
tf.keras.models.save_model(model, keras_file, include_optimizer=False)

Baseline test accuracy: 0.9445064067840576
Saving model to:  /tmp/tmpouxz5ow5.h5


In [13]:
import tensorflow_model_optimization as tfmot

cluster_weights = tfmot.clustering.keras.cluster_weights
CentroidInitialization = tfmot.clustering.keras.CentroidInitialization

clustering_params = {
  'number_of_clusters': 16,
  'cluster_centroids_init': CentroidInitialization.LINEAR
}

# Cluster a whole model
clustered_model = cluster_weights(model, **clustering_params)

# Use smaller learning rate for fine-tuning clustered model
opt = tf.keras.optimizers.Adam(learning_rate=1e-5)

clustered_model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=opt,
  metrics=['accuracy'])

clustered_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cluster_reshape (ClusterWeig (None, 28, 28, 1)         0         
_________________________________________________________________
cluster_conv2d (ClusterWeigh (None, 28, 28, 16)        320       
_________________________________________________________________
cluster_conv2d_1 (ClusterWei (None, 28, 28, 32)        36912     
_________________________________________________________________
cluster_conv2d_2 (ClusterWei (None, 28, 28, 64)        589904    
_________________________________________________________________
cluster_max_pooling2d (Clust (None, 14, 14, 64)        0         
_________________________________________________________________
cluster_flatten (ClusterWeig (None, 12544)             0         
_________________________________________________________________
cluster_dense (ClusterWeight (None, 1024)              2

In [14]:
# Fine-tune model
clustered_model.fit(
  x_train,
  y_train,
  batch_size=500,
  epochs=1,
  validation_data=(x_test,y_test))

/usr/local/lib/python3.7/dist-packages/keras/backend.py:4907: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  '"`sparse_categorical_crossentropy` received `from_logits=True`, but '


55/55 [==============================] - 8s 124ms/step - loss: 2.3871e-07 - accuracy: 1.0000 - val_loss: 0.3319 - val_accuracy: 0.9453


In [15]:
_, clustered_model_accuracy = clustered_model.evaluate(
  x_test, y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Clustered test accuracy:', clustered_model_accuracy)

Baseline test accuracy: 0.9445064067840576
Clustered test accuracy: 0.945343017578125


In [16]:
final_model = tfmot.clustering.keras.strip_clustering(clustered_model)

_, clustered_keras_file = tempfile.mkstemp('.h5')
print('Saving clustered model to: ', clustered_keras_file)
tf.keras.models.save_model(final_model, clustered_keras_file, 
                           include_optimizer=False)

Saving clustered model to:  /tmp/tmpm5s11vrq.h5


In [17]:
clustered_tflite_file = '/tmp/clustered_mnist.tflite'
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
tflite_clustered_model = converter.convert()
with open(clustered_tflite_file, 'wb') as f:
  f.write(tflite_clustered_model)
print('Saved clustered TFLite model to:', clustered_tflite_file)

INFO:tensorflow:Assets written to: /tmp/tmpzn8bh8gq/assets
Saved clustered TFLite model to: /tmp/clustered_mnist.tflite


In [18]:
def get_gzipped_model_size(file):
  # It returns the size of the gzipped model in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [19]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered Keras model: %.2f bytes" % (get_gzipped_model_size(clustered_keras_file)))
print("Size of gzipped clustered TFlite model: %.2f bytes" % (get_gzipped_model_size(clustered_tflite_file)))

Size of gzipped baseline Keras model: 48779626.00 bytes
Size of gzipped clustered Keras model: 3192896.00 bytes
Size of gzipped clustered TFlite model: 3317693.00 bytes


In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(final_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

_, quantized_and_clustered_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_clustered_tflite_file, 'wb') as f:
  f.write(tflite_quant_model)

print('Saved quantized and clustered TFLite model to:', quantized_and_clustered_tflite_file)
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped clustered and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_clustered_tflite_file)))

INFO:tensorflow:Assets written to: /tmp/tmp_uiy_aa3/assets


INFO:tensorflow:Assets written to: /tmp/tmp_uiy_aa3/assets


Saved quantized and clustered TFLite model to: /tmp/tmp3ogevdag.tflite
Size of gzipped baseline Keras model: 48779626.00 bytes
Size of gzipped clustered and quantized TFlite model: 2445806.00 bytes


In [23]:
def eval_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_digits = []
  for i, test_image in enumerate(x_test):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == y_test).mean()
  return accuracy

In [24]:
interpreter = tf.lite.Interpreter(model_content=tflite_quant_model)
interpreter.allocate_tensors()

test_accuracy = eval_model(interpreter)

print('Clustered and quantized TFLite test_accuracy:', test_accuracy)
print('Clustered TF test accuracy:', clustered_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.


Clustered and quantized TFLite test_accuracy: 0.9452035694366983
Clustered TF test accuracy: 0.945343017578125
